In [15]:
from pymongo import MongoClient
import pandas as pd
from pandas.io.json import json_normalize
from geojson import Point
client = MongoClient().Test
db = client.Cleaned_companies

In [16]:
def geoQueryNear(lat, lng, distance=10000):
    return {
        "location": {
            "$near": { 
                "$geometry": {
                        "type": "Point", 
                        "coordinates": [lng,lat]},
                    "$maxDistance":distance,
                },
            }
        }

In [18]:
doc = db.find(projection={"_id":0})
df = json_normalize(list(doc))
df = df.drop("location.type",axis=1)
df["long"] = [a[0] for a in df["location.coordinates"]]
df["lat"] = [a[1] for a in df["location.coordinates"]]

In [19]:
dfq = df[["lat","long"]]
def cercania (x,y,radio):
    return list(db.find(geoQueryNear(x,y,radio),projection={"name":1,"number_of_employees":1,"_id":0}))

In [20]:
def query_final(radio):
    busqueda = dfq.apply(lambda x: cercania(x[0],x[1],radio),axis=1)
    vaciatotal = []
    for fila in busqueda[:]:
        vacia = []
        for elemento in fila:   
            vacia.append(elemento["number_of_employees"])
        vaciatotal.append(sum(vacia))
    df["Near_Emp"] = vaciatotal
    df3 = df.sort_values(by="Near_Emp",ascending=False).head(10)
    return df3

In [24]:
# Radio 100 mts
radio_100 = query_final(100)
data = radio_100.to_csv('radio100.csv', index=False)

In [26]:
# radio 1.000 mts
radio_1000 = query_final(1000)
data = radio_1000.to_csv('radio_1000.csv', index=False)

In [28]:
# radio 10.000 mts
radio_10k=query_final(10000)
data = radio_10k.to_csv('radio10k.csv', index=False)

In [27]:
radio_1000 

,location.coordinates,name,number_of_employees,long,lat,Near_Emp
777,"[-73.723999, 41.109534]",IBM,388000,-73.723999,41.109534,388000
2608,"[-121.934573, 37.293687]",Telera,100,-121.934573,37.293687,315570
133,"[-121.935688, 37.288974]",Hightail,200,-121.935688,37.288974,315520
2950,"[-121.933651, 37.291228]",Asyncast Corporation,20,-121.933651,37.291228,315490
4187,"[-121.93418, 37.291065]",tempCFO Solutions,22,-121.934180,37.291065,315490
135,"[-121.927696, 37.294465]",PayPal,300000,-121.927696,37.294465,315470
466,"[-121.9317853, 37.2906027]",clipsync,28,-121.931785,37.290603,315470
11,"[-121.930035, 37.295005]",eBay,15000,-121.930035,37.295005,315470
3767,"[-121.9303616, 37.3015115]",Naseeb Networks,100,-121.930362,37.301511,315200
116,"[-95.712891, 37.09024]",CollegeWikis,7,-95.712891,37.090240,190054
